# NEISS Public Data Reports - Initial Analysis

## Configuration

In [1]:
import re
import string
from zipfile import ZipFile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Data

In [2]:
zf = ZipFile('../data/neiss-public/neiss_tsv.zip')
filenames = [file.filename for file in zf.infolist() if "MACOS" not in file.filename]
data={}
for file in filenames:
    print(file)
    year = file[-8:-4]
    try:
        data[int(year)] = pd.read_csv(zf.open(file), delimiter='\t', low_memory=False, encoding='utf-8')
    except:
        print('\t Error in file')
        data[int(year)] = pd.read_csv(zf.open(file), delimiter='\t', low_memory=False, encoding='ISO-8859-1')

neiss2011.tsv
neiss2012.tsv
neiss2013.tsv
neiss2014.tsv
neiss2015.tsv
neiss2016.tsv
neiss2017.tsv
	 Error in file
neiss2018.tsv
neiss2019.tsv
neiss2020.tsv


## Analysis

### Basic Table Info

In [3]:
def race_cols(cols):
    
    race_cols = [x for x in cols if 'race' in x.lower()]
    
    if len(race_cols) == 0:
        return 'None'
    
    race_str = race_cols[0]
    for c in race_cols[1:]:
        race_str += ', ' + c
        
    return race_str


def eth_cols(cols):
    
    eth_cols = [x for x in cols if (('ethnicity' in x.lower()) | ('hispanic' in x.lower()) | ('latino' in x.lower()))]
    
    if len(eth_cols) == 0:
        return 'None'
    
    eth_str = eth_cols[0]
    for c in eth_cols[1:]:
        eth_str += ', ' + c
        
    return eth_str

In [4]:
for year in data.keys():
    print('Year: {}, Entry Count: {}, Column Count: {}, Race Cols: {}, Ehtnicity Cols: {}'\
          .format(year, len(data[year]), len(data[year].columns.tolist()), race_cols(cols=data[year].columns.tolist()), eth_cols(cols=data[year].columns.tolist())))

Year: 2011, Entry Count: 396502, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2012, Entry Count: 394383, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2013, Entry Count: 376927, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2014, Entry Count: 367492, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2015, Entry Count: 359129, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2016, Entry Count: 375196, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2017, Entry Count: 386907, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2018, Entry Count: 361668, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2019, Entry Count: 358716, Column Count: 25, Race Cols: Race, Other_Race, Ehtnicity Cols: Hispanic
Year: 2020, Entry Count: 309370, Column Count: 25, Race

### Combine All Data

In [5]:
df = pd.concat(list(data.values()))
del(data)
df.head()

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,Other_Diagnosis,...,Product_1,Product_2,Product_3,Alcohol,Drug,Narrative,Stratum,PSU,Weight,Narrative_1
0,110100393,01/01/2011,84,2.0,1.0,NaN,NaN,76.0,59.0,NaN,...,4076.0,0.0,0.0,NaN,NaN,"84YOF, FELL FROM BED AT HOME, LACERATION TO FO...",S,71.0,68.3455,NaN
1,110100394,01/01/2011,26,2.0,2.0,NaN,NaN,92.0,64.0,NaN,...,3299.0,0.0,0.0,NaN,NaN,"26YOF, FELL DOWN AT HOME WHILE RUNNING, SPRAIN...",S,71.0,68.3455,NaN
2,110100396,01/01/2011,97,2.0,1.0,NaN,NaN,75.0,53.0,NaN,...,1807.0,0.0,0.0,NaN,NaN,"97YOF, FELL AND HIT HEAD ON FLOOR, CONTUSION T...",S,71.0,68.3455,NaN
3,110100397,01/02/2011,38,2.0,1.0,NaN,NaN,79.0,64.0,NaN,...,1658.0,127.0,0.0,NaN,NaN,"38YOF, GETTING CLOTHES OUT OF DRYER AT HOME, L...",S,71.0,68.3455,NaN
4,110100398,01/02/2011,48,2.0,1.0,NaN,NaN,79.0,71.0,PAIN,...,550.0,0.0,0.0,NaN,NaN,"48YOF, TRIPPED OVER PHONE CORD AT HOME, LOWER ...",S,71.0,68.3455,NaN


### Check Race and Other Race Count

In [6]:
df['Race'].value_counts()

1.0    1653724
0.0    1225637
2.0     540051
3.0     207477
4.0      45555
5.0      10411
6.0       3431
Name: Race, dtype: int64

In [7]:
df['Other_Race'].value_counts()

HISPANIC         157654
UNKNOWN           23702
MULTI-RACIAL      19119
HISP              15579
NS                 3514
                  ...  
HIISP                 1
MULTI  RACIAL         1
HIISPANIC             1
MULTI RACIA.          1
UNKNOWN/WHITE         1
Name: Other_Race, Length: 469, dtype: int64

In [8]:
df['Hispanic'].value_counts()

2.0    372383
0.0    236149
1.0     59553
Name: Hispanic, dtype: int64

In [9]:
df[['Race', 'Other_Race']].groupby(['Race', 'Other_Race']).size().unstack().T

Race,0.0,1.0,2.0,3.0,4.0,5.0,6.0
Other_Race,,,,,,,
"""",1.0,1.0,NaN,NaN,NaN,NaN,NaN
"""OTHER""",NaN,NaN,NaN,4.0,NaN,NaN,NaN
-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
- LAC. TO SCALP,NaN,NaN,NaN,1.0,NaN,NaN,NaN
.,NaN,2.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
`,21.0,88.0,10.0,NaN,2.0,NaN,NaN
`HISP,NaN,NaN,NaN,2.0,NaN,NaN,NaN
`HISPANIC,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [10]:
df[['Race', 'Other_Race', 'Hispanic']].groupby(['Race', 'Other_Race', 'Hispanic']).size().unstack()

Hispanic              0.0  1.0   2.0
Race Other_Race                     
3.0  1&2              1.0  2.0   4.0
     2&6              NaN  NaN   1.0
     AFRICAN          NaN  NaN  19.0
     ARGENTINIAN      NaN  1.0   NaN
     ASIAN/WHITE      NaN  NaN   2.0
...                   ...  ...   ...
     UNKNOWN/WHITE    NaN  NaN   1.0
     WHITE AND BLACK  NaN  NaN   1.0
     WHITE, BLACK     NaN  NaN   1.0
     WHITE/BLACK/AFR  NaN  NaN   1.0
     WHITE/NATIVE AM  NaN  NaN   1.0

[101 rows x 3 columns]

### Empty Values

In [11]:
len(df) - len(df.dropna(subset=['Race', 'Other_Race', 'Hispanic'], how='all'))

4

In [12]:
df['Race'].isnull().value_counts()

False    3686286
True           4
Name: Race, dtype: int64

In [13]:
df['Other_Race'].isnull().value_counts()

True     3458914
False     227376
Name: Other_Race, dtype: int64

In [14]:
df['Hispanic'].isnull().value_counts()

True     3018205
False     668085
Name: Hispanic, dtype: int64

## Preprocessing

### Convert Race and Hispanic from Code to Label

In [16]:
lookup = pd.read_excel('xlsx/neiss2020.xlsx', sheet_name=1)
lookup.replace('               .', np.nan, inplace=True)
lookup['Starting value for format'] = lookup['Starting value for format'].astype(float)

In [17]:
lookup[lookup['Format name']=='HISP']

,Format name,Starting value for format,Ending value for format,Format value label
105,HISP,NaN,NaN,NA before 2019
106,HISP,0.0,0,Unk/Not stated
107,HISP,1.0,1,Yes
108,HISP,2.0,2,No


In [18]:
df['Hispanic'] = df['Hispanic'].map({1.0:'Yes', 2.0:'No'})
df.Hispanic.value_counts()

No     372383
Yes     59553
Name: Hispanic, dtype: int64

In [57]:
race_lookup

,Format name,Starting value for format,Ending value for format,Format value label
1242,RACE,0.0,0,N.S.
1243,RACE,1.0,1,WHITE
1244,RACE,2.0,2,BLACK/AFRICAN AMERICAN
1245,RACE,3.0,3,OTHER
1246,RACE,4.0,4,ASIAN
1247,RACE,5.0,5,AMERICAN INDIAN/ALASKA NATIVE
1248,RACE,6.0,6,NATIVE HAWAIIAN/PACIFIC ISLANDER


In [19]:
race_lookup = lookup[lookup['Format name']=='RACE']
df['Race'] = df['Race'].map({code:label for code,label in zip(race_lookup['Starting value for format'], race_lookup['Format value label'])})
df.Race.value_counts()

WHITE                               1653724
N.S.                                1225637
BLACK/AFRICAN AMERICAN               540051
OTHER                                207477
ASIAN                                 45555
AMERICAN INDIAN/ALASKA NATIVE         10411
NATIVE HAWAIIAN/PACIFIC ISLANDER       3431
Name: Race, dtype: int64

### Other Race Clean Up

In [20]:
# Get Unique Other Race 
oRace = df['Other_Race'].value_counts().reset_index(drop=False)
oRace.columns = ['Other_Race', 'Count']
print(len(oRace)) 
oRace.head()

469


,Other_Race,Count
0,HISPANIC,157654
1,UNKNOWN,23702
2,MULTI-RACIAL,19119
3,HISP,15579
4,NS,3514


In [21]:
# Lower Case
oRace['LCRace'] = [str(x).lower() for x in oRace['Other_Race']]
oRace['LCRace'].nunique()

469

In [22]:
# Remove Digits
oRace['NoNumbsRace'] = [re.sub(r"\d+", "", x) for x in oRace['LCRace']]
oRace['NoNumbsRace'].nunique()

448

In [23]:
# Remove Punctuation/Special Characters
oRace['NoPuncRace'] = [re.sub(r'[^\w\s]+', ' ', x) for x in oRace['NoNumbsRace']]
oRace['NoPuncRace'].nunique()

431

In [24]:
# Strip Whitespace
oRace['StrippedRace'] = oRace['NoPuncRace'].str.strip()
oRace['StrippedRace'].nunique()

421

In [25]:
# map(lambda x: x.isalnum(), oRace['ProcessedRace'].astype(str))

In [26]:
oRace['ProcessedRace'] = oRace['StrippedRace']
oRace['str_len'] = [len(x) for x in oRace['ProcessedRace']] 
oRace['alphanum'] =  oRace['ProcessedRace'].astype(str).apply(lambda x: x.isalnum())

In [27]:
oRace['alphanum'].value_counts()

True     309
False    160
Name: alphanum, dtype: int64

In [28]:
oRace.head()

,Other_Race,Count,LCRace,NoNumbsRace,NoPuncRace,StrippedRace,ProcessedRace,str_len,alphanum
0,HISPANIC,157654,hispanic,hispanic,hispanic,hispanic,hispanic,8,True
1,UNKNOWN,23702,unknown,unknown,unknown,unknown,unknown,7,True
2,MULTI-RACIAL,19119,multi-racial,multi-racial,multi racial,multi racial,multi racial,12,False
3,HISP,15579,hisp,hisp,hisp,hisp,hisp,4,True
4,NS,3514,ns,ns,ns,ns,ns,2,True


In [29]:
def preprocess_text(word):
    if str(word)=='nan':
        return np.nan
    else:
        # lower case
        word = word.lower()
        # remove digits
        word = re.sub(r'\d+', '', word)
        # remove punctuation and special chars
        word = re.sub(r'[^\w\s]+', ' ', word)
        # strip whitespace
        word = word.strip()
        
        return word

In [30]:
df['Other_Race'].apply(preprocess_text).value_counts(dropna=False)

NaN               3458914
hispanic           157684
unknown             23702
multi racial        19468
hisp                15582
                   ...   
african somali          1
hiso                    1
hispanic s p b          1
qryomhispanic           1
unknown white           1
Name: Other_Race, Length: 422, dtype: int64

### Fuzzy Matching on Other Race

In [31]:
# # fuzzywuzzy package has been moved to thefuzz
# # https://github.com/seatgeek/thefuzz
# from fuzzywuzzy import fuzz
# import fuzzywuzzy.process as fuzz_process

In [32]:
from thefuzz import fuzz
import thefuzz.process as fuzz_process

In [33]:
oRacesL = oRace.loc[((oRace['ProcessedRace']  != '') & (oRace['ProcessedRace']  != '``') & (oRace['str_len']  >= 2))]['ProcessedRace'].unique()

race_match = {}
for race in oRacesL:
    race_match[race] = []
    for race1 in oRacesL:
        res = fuzz_process.extract(race, [race1], scorer=fuzz.token_sort_ratio, limit=1)
        race_match[race].append(res[0][1])    

race_match = pd.DataFrame.from_dict(race_match, orient='index', columns = oRacesL)

In [34]:
race_match

,hispanic,unknown,multi racial,hisp,ns,somali,unk,unkn,multiracial,biracial,...,haitian,hmong,multiple racial,indian black,hispanie,uinknown,wolof,nispanic,american samoa,unknown white
hispanic,100,13,30,67,20,43,18,17,32,38,...,53,31,35,30,88,25,0,88,27,19
unknown,13,100,11,0,22,15,60,73,11,0,...,14,33,9,32,13,93,17,27,19,70
multi racial,30,11,100,12,0,33,13,12,96,70,...,32,12,89,42,30,20,12,30,38,24
hisp,67,0,12,100,33,20,0,0,13,17,...,36,22,21,12,67,17,0,50,22,24
ns,20,22,0,33,100,25,40,33,0,0,...,22,29,0,14,20,20,0,40,25,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uinknown,25,93,20,17,20,14,55,67,21,12,...,27,31,17,30,25,100,15,25,27,67
wolof,0,17,12,0,0,36,0,0,12,15,...,0,20,10,12,0,15,100,0,11,11
nispanic,88,27,30,50,40,43,18,33,32,38,...,40,15,35,40,75,25,0,100,27,29
american samoa,27,19,38,22,25,30,12,11,32,27,...,38,21,34,31,27,27,11,27,100,15


In [35]:
race_match.loc[race_match['hispanic']>70].index.values

array(['hispanic', 'hisapnic', 'hispainc', 'hsipanic', 'hipanic',
       'non hispanic', 'hispanc', 'hspanic', 'hispainic', 'hispanci',
       'hipsanic', 'hisapanic', 'hisipanic', 'hispanic  pr', 'hispani',
       'hispnic', 'hidpanic', 'hispnaic', 'hispaic', 'haspanic',
       'hispanice', 'hispaniic', 'hispnanic', 'hisanic', 'hiapanic',
       'hispaanic', 'hisoanic', 'hispsanic', 'hispanich', 'nonhispanic',
       'hispsnic', 'hispoanic', 'hispanicx', 'hisspanic', 'hhispanic',
       'hisppanic', 'hispaniv', 'hispan', 'and hispanic', 'his panic',
       'hispapnic', 'ispanic', 'hispain', 'hispaini', 'spanic',
       'bispanic', 'hipani', 'hispaninc', 'hispainic  la',
       'hispanic  mexic', 'black hispanic', 'black  hispanic',
       'hispanic black', 'hispanic  domin', 'hs panic', 'gispanic',
       'hispa nic', 'hispabnic', 'hyspanic', 'hispanic latin',
       'hispanica', 'hisdpanic', 'huispanic', 'hispanid', 'hispanicq',
       'hsispanic', 'hipspanic', 'blk hispanic', 'hiisp

In [36]:
[w for w in race_match.loc[race_match['hispanic']>70].index.values if 'no' in w]

['non hispanic', 'nonhispanic']

In [37]:
hisp_match = race_match.loc[race_match['hispanic']>70].index.values
hisp_match = np.array([match for match in hisp_match if "no" not in match])
hisp_match

array(['hispanic', 'hisapnic', 'hispainc', 'hsipanic', 'hipanic',
       'hispanc', 'hspanic', 'hispainic', 'hispanci', 'hipsanic',
       'hisapanic', 'hisipanic', 'hispanic  pr', 'hispani', 'hispnic',
       'hidpanic', 'hispnaic', 'hispaic', 'haspanic', 'hispanice',
       'hispaniic', 'hispnanic', 'hisanic', 'hiapanic', 'hispaanic',
       'hisoanic', 'hispsanic', 'hispanich', 'hispsnic', 'hispoanic',
       'hispanicx', 'hisspanic', 'hhispanic', 'hisppanic', 'hispaniv',
       'hispan', 'and hispanic', 'his panic', 'hispapnic', 'ispanic',
       'hispain', 'hispaini', 'spanic', 'bispanic', 'hipani', 'hispaninc',
       'hispainic  la', 'hispanic  mexic', 'black hispanic',
       'black  hispanic', 'hispanic black', 'hispanic  domin', 'hs panic',
       'gispanic', 'hispa nic', 'hispabnic', 'hyspanic', 'hispanic latin',
       'hispanica', 'hisdpanic', 'huispanic', 'hispanid', 'hispanicq',
       'hsispanic', 'hipspanic', 'blk hispanic', 'hiispanic', 'hispanix',
       'hispaniiic'

In [38]:
[x for x in race_match.index if 'his' in x and x not in hisp_match and 'no' not in x]

['hisp',
 'his',
 'hiswp',
 'hisp anic',
 'hispanic mexica',
 'his anic',
 'hisdp',
 'hispanicmjjjjjj',
 'hispanic puerto',
 'hiso']

In [39]:
latino_match = race_match.loc[race_match['latino']>70].index.values
latino_match

array(['latino', 'latin', 'latvian'], dtype=object)

In [40]:
race_match.loc[race_match['multiracial']>70].index.values

array(['multi racial', 'multiracial', 'biracial', 'multracial',
       'mult racial', 'multi race', 'bi multiracial', 'multirace',
       'multi racia', 'multil racial', 'multi  racial', 'multiracail',
       'multo racial', 'multiple racial'], dtype=object)

In [41]:
[x for x in race_match.index if 'more' in x]

['two more races', 'more races', 'two or more rac']

In [42]:
multiracial_match = race_match.loc[race_match['multiracial']>70].index.values
multiracial_match = np.append(multiracial_match, [x for x in race_match.index if 'more' in x])
multiracial_match

array(['multi racial', 'multiracial', 'biracial', 'multracial',
       'mult racial', 'multi race', 'bi multiracial', 'multirace',
       'multi racia', 'multil racial', 'multi  racial', 'multiracail',
       'multo racial', 'multiple racial', 'two more races', 'more races',
       'two or more rac'], dtype=object)

In [43]:
unk_match = race_match.loc[race_match['unknown']>70].index.values
unk_match

array(['unknown', 'unkn', 'uknown', 'unknwon', 'unkown', 'unknonw',
       'unknokwn', 'unnown', 'ukknown', 'sunknown', 'unknonwn', 'unknowm',
       'unwknown', 'unknow n', 'unknow', 'unknowmn', 'unknkown',
       'unkinown', 'uniknown', 'unknnown', 'uinknown'], dtype=object)

In [44]:
decline_match = race_match.loc[race_match['decline']>55].index.values
decline_match

array(['declined', 'decline', 'chinese', 'pt declined', 'declined to ans',
       'declined to lis', 'refused decline', 'declines to lis',
       'family declined'], dtype=object)

### Move Hispanic/Latino from `Other_Race` to `Hispanic`

In [45]:
# first, find values of Hispanic when Other_Race contains hispanic_matches
df_abr = df[['Race', 'Other_Race', 'Hispanic']].copy()
df_abr.head()

,Race,Other_Race,Hispanic
0,WHITE,NaN,NaN
1,ASIAN,NaN,NaN
2,WHITE,NaN,NaN
3,WHITE,NaN,NaN
4,WHITE,NaN,NaN


In [46]:
df_abr['Other_Race_Preprocessed'] = df_abr['Other_Race'].apply(preprocess_text)
df_abr['Other_Race_Preprocessed'].value_counts()

hispanic          157684
unknown            23702
multi racial       19468
hisp               15582
ns                  3514
                   ...  
african somali         1
hiso                   1
hispanic s p b         1
qryomhispanic          1
unknown white          1
Name: Other_Race_Preprocessed, Length: 421, dtype: int64

In [47]:
df_abr[df_abr.Other_Race_Preprocessed.isin(hisp_match)].Hispanic.value_counts(dropna=False)

NaN    156384
Yes      1658
Name: Hispanic, dtype: int64

In [48]:
df_abr.Hispanic.value_counts(dropna=False)/len(df_abr)

NaN    0.882826
No     0.101018
Yes    0.016155
Name: Hispanic, dtype: float64

In [49]:
# if other_race (after preprocessing) contains any of the hispanic_matches
# change Hispanic to 'Yes'
df_abr['Hispanic_filled'] = np.where(df_abr.Other_Race_Preprocessed.isin(hisp_match), 'Yes', df_abr.Hispanic)
df_abr.Hispanic_filled.value_counts(dropna=False)/len(df_abr)

NaN    0.840403
No     0.101018
Yes    0.058578
Name: Hispanic_filled, dtype: float64

In [50]:
# do the same with 'latino'
latino_match

array(['latino', 'latin', 'latvian'], dtype=object)

In [51]:
df_abr[df_abr.Other_Race_Preprocessed=='latino'].Hispanic.value_counts(dropna=False)

Yes    80
NaN     1
Name: Hispanic, dtype: int64

In [52]:
df_abr['Hispanic_filled'] = np.where(df_abr.Other_Race_Preprocessed=='latino', 'Yes', df_abr.Hispanic_filled)

df_abr[df_abr.Other_Race_Preprocessed=='latino'].Hispanic_filled.value_counts(dropna=False)

Yes    81
Name: Hispanic_filled, dtype: int64

#### Remove Hispanic/Latino from `Other_Race`

In [53]:
df_abr['Other_Race_hisp_removed'] = df_abr['Other_Race_Preprocessed'].replace('|'.join(hisp_match), np.nan, regex=True)
df_abr['Other_Race_hisp_removed'].value_counts(dropna=False)

NaN              3617015
unknown            23702
multi racial       19468
hisp               15582
ns                  3514
                  ...   
haitian                1
someli                 1
r wrist                1
nss                    1
unknown white          1
Name: Other_Race_hisp_removed, Length: 338, dtype: int64

### Clean up other responses

In [54]:
multiracial_match

array(['multi racial', 'multiracial', 'biracial', 'multracial',
       'mult racial', 'multi race', 'bi multiracial', 'multirace',
       'multi racia', 'multil racial', 'multi  racial', 'multiracail',
       'multo racial', 'multiple racial', 'two more races', 'more races',
       'two or more rac'], dtype=object)

In [55]:
unk_match

array(['unknown', 'unkn', 'uknown', 'unknwon', 'unkown', 'unknonw',
       'unknokwn', 'unnown', 'ukknown', 'sunknown', 'unknonwn', 'unknowm',
       'unwknown', 'unknow n', 'unknow', 'unknowmn', 'unknkown',
       'unkinown', 'uniknown', 'unknnown', 'uinknown'], dtype=object)

In [56]:
decline_match

array(['declined', 'decline', 'chinese', 'pt declined', 'declined to ans',
       'declined to lis', 'refused decline', 'declines to lis',
       'family declined'], dtype=object)

In [70]:
df[df.Age>150].shape[0]/len(df)

0.05937650049236495